In [1]:
import folium

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from folium.plugins import MarkerCluster


In [3]:
m = folium.Map(location=[51.5, -0.1], zoom_start=15)

In [4]:
m

In [5]:
accident_data = pd.read_csv("dataWith5000RNRecordsWithIDAndHeaders")
accident_data = accident_data.sample(n=5000, random_state=42)
accident_data.dropna(subset=["Latitude", "Longitude"], inplace=True)

In [6]:
accident_data.head(5)

,Accident_ID,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
1501,1502,635540,170520,1.383795,51.384758,46,3,2,1,06-11-2014,...,0,0,1,1,2,0,0,1,1,E01024678
2586,2587,435753,389544,-1.463722,53.401504,14,3,2,1,21-01-2014,...,0,0,1,1,1,0,0,1,1,E01007870
2653,2654,383204,351878,-2.252102,53.063861,21,3,2,1,04-12-2014,...,0,0,4,1,1,0,0,2,1,E01029596
1055,1056,531735,104993,-0.130808,50.829425,47,3,3,2,15-03-2014,...,0,0,1,9,1,0,0,1,2,E01016894
705,706,434222,399622,-1.485656,53.492189,14,3,2,1,25-09-2014,...,0,0,1,1,1,0,0,2,1,E01007424


In [7]:
accident_data.shape

(5000, 32)

In [8]:
def get_geojson_grid(upper_right, lower_left, n=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [9]:
 def generate_heat_gridmap(accident_data, htmlfilename):
    m = folium.Map(zoom_start = 5, location=[55, 0])

    # Generate GeoJson grid
    top_right = [58, 2]
    top_left = [49, -8]

    grid = get_geojson_grid(top_right, top_left, n=6)

    # Calculate exposures in grid
    regional_counts = []

    for box in grid:
        upper_right = box["properties"]["upper_right"]
        lower_left = box["properties"]["lower_left"]

        mask = (
            (accident_data.Latitude != upper_right[1]) & (accident_data.Latitude != lower_left[1]) &
            (accident_data.Longitude != upper_right[0]) & (accident_data.Longitude != lower_left[0])
               )

        region_incidents = len(accident_data[mask])
        regional_counts.append(region_incidents)


    worst_region = max(regional_counts)

    # Add GeoJson to map
    for i, box in enumerate(grid):
        geo_json = json.dumps(box)

        color = plt.cm.Reds(regional_counts[i] / worst_region)
        color = mpl.colors.to_hex(color)

        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature, color=color: {
                                                                            'fillColor': color,
                                                                            'color':"black",
                                                                            'weight': 2,
                                                                            'dashArray': '5, 5',
                                                                            'fillOpacity': 0.55,
                                                                        })

        m.add_child(gj)

    # Marker clusters
    locations = list(zip(accident_data.Latitude, accident_data.Longitude))
    icons = [folium.Icon(icon="car", prefix="fa") for _ in range(len(locations))]

    # Create popups
    popup_content = []
    for incident in accident_data.itertuples():
        light_conditions = "Light Condition: {} <br/>".format(incident.Light_Conditions)
        triangle_count = "Triangle Count: {} ".format(incident.Triangle_Count)
        content = light_conditions + triangle_count
        popup_content.append(content)

    popups = [folium.Popup(content) for content in popup_content]

    cluster = MarkerCluster(locations=locations, icons=icons, popups=popups)
    m.add_child(cluster)

    m.save(htmlfilename)

In [10]:
Light_100_data = pd.read_csv("Light_100_TriangleCount.csv")
Light_100_data.head(5)

,Node,Triangle_Count
0,57,2
1,59,2
2,61,2
3,63,2
4,65,2


In [11]:
Light_500_data = pd.read_csv("Light_500_TriangleCount.csv")
Light_500_data.head(5)

,Node,Triangle_Count
0,497,5575
1,499,5575
2,489,5566
3,491,5566
4,493,5566


In [12]:
Light_1000_data = pd.read_csv("Light_1000_TriangleCount.csv")
Light_1000_data.head(5)

,Node,Triangle_Count
0,999,51761
1,997,51726
2,993,51725
3,995,51725
4,985,51698


In [13]:
required_columns = list(accident_data.columns) + ['Triangle_Count']

In [14]:
accident_light_100_data = pd.merge(accident_data, Light_100_data, left_on='Accident_ID', right_on='Node', how='inner')

In [15]:
accident_light_100 = accident_light_100_data[required_columns].sort_values('Triangle_Count', ascending=False)

In [16]:
accident_light_100_top5 = accident_light_100.head(5)
accident_light_100.head(5)

,Accident_ID,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Triangle_Count
50,79,424652,322304,-1.635825,52.797716,21,3,2,2,09-11-2014,...,0,1,1,1,0,0,1,1,E01032898,2
92,67,282430,212650,-3.706464,51.800057,63,2,1,1,18-03-2014,...,0,1,1,1,0,0,2,1,W01000496,2
33,57,537520,182090,-0.019237,51.520940,1,3,1,1,30-01-2014,...,1,4,1,2,0,0,1,1,E01004242,2
35,59,319709,567613,-3.256611,54.996620,98,2,1,1,11-04-2014,...,0,1,1,1,0,0,2,1,NaN,2
36,77,359095,427377,-2.621672,53.741157,4,3,1,2,28-11-2014,...,0,4,1,2,0,0,1,1,E01025414,2


In [17]:
generate_heat_gridmap(accident_light_100_top5, "car_accidents_light_100_v1.html")

In [18]:
accident_light_500_data = pd.merge(accident_data, Light_500_data, left_on='Accident_ID', right_on='Node', how='inner')

In [19]:
accident_light_500 = accident_light_500_data[required_columns].sort_values('Triangle_Count', ascending=False)

In [20]:
accident_light_500_top5 = accident_light_500.head(5)
accident_light_500.head(5)

,Accident_ID,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Triangle_Count
269,497,499545,149713,-0.575444,51.237866,45,3,2,1,25-11-2014,...,0,1,1,2,0,0,1,2,E01030451,5575
173,499,246167,55753,-4.165114,50.381429,50,3,1,1,21-03-2014,...,4,1,1,1,0,0,1,2,E01015170,5575
490,493,385477,285029,-2.215200,52.462980,21,2,1,1,27-04-2014,...,0,4,1,1,0,7,2,2,E01029651,5566
482,489,510510,198440,-0.403223,51.673820,41,3,2,1,26-09-2014,...,5,1,1,1,0,0,1,1,E01023869,5566
135,495,418756,293399,-1.724906,52.538103,23,3,2,1,05-08-2014,...,0,4,1,1,0,0,2,1,E01031023,5566


In [21]:
generate_heat_gridmap(accident_light_500_top5, "car_accidents_light_500_v1.html")

In [22]:
accident_light_1000_data = pd.merge(accident_data, Light_1000_data, left_on='Accident_ID', right_on='Node', how='inner')

In [23]:
accident_light_1000 = accident_light_1000_data[required_columns].sort_values('Triangle_Count', ascending=False)

In [24]:
accident_light_1000_top5 = accident_light_1000.head(5)
accident_light_1000.head(5)

,Accident_ID,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Triangle_Count
561,999,394486,403791,-2.084662,53.530739,6,3,2,1,14-01-2014,...,0,1,1,2,0,0,1,1,E01005383,51761
120,997,265896,664654,-4.144004,55.856414,97,2,1,3,27-12-2014,...,0,4,1,2,0,0,1,1,NaN,51726
70,995,568015,190027,0.423713,51.584001,42,3,1,1,02-03-2014,...,0,6,8,2,0,0,1,1,E01021261,51725
849,993,488313,169020,-0.731515,51.413278,43,3,2,1,12-10-2014,...,0,1,1,1,0,0,1,2,E01032728,51725
839,985,432552,433926,-1.507245,53.800621,13,3,2,1,10-09-2014,...,0,1,1,1,0,0,1,1,E01011348,51698


In [25]:
generate_heat_gridmap(accident_light_1000_top5, "car_accidents_light_1000_v1.html")